# SET

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
def carregar_arquivo_excel(caminho, sheet_name=0, skiprows=None):
    return pd.read_excel(caminho, sheet_name=sheet_name, skiprows=skiprows)

caminhos = {
'caminho_pccc_20_25' : r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\PCCC_20_25.XLSX",
'caminho_pccc_15_19':  r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\PCCC_15_19.XLSX",
'caminho_pccc_09_14' : r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\PCCC_09_14.XLSX",
'caminho_pccc_06_08' : r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\PCCC_06_08.XLSX",
}





In [ ]:
def limpar_colunas(df, colunas_para_remover, nova_index=None, renomear_colunas=None):
    df = df.drop(columns=colunas_para_remover, errors='ignore')
    df = df.dropna()
    if renomear_colunas:
        df = df.rename(columns=renomear_colunas)
    if nova_index:
        df = df.set_index(nova_index)
    return df

In [ ]:
def preparar_dados_lstm(serie, n_steps):
    """
        Transforma uma série temporal em um conjunto de dados supervisionado para treino de LSTM,
    usando o padrão de janela deslizante (sliding window).

    Parâmetros:
    
    serie : array-like de shape (n_amostras,) ou (n_amostras, 1)
        Série temporal (normalizada) em que cada elemento é uma observação sequencial.
    n_steps : int
        Número de passos de tempo que formam cada janela de entrada.

    Retorna:
    
    X : np.ndarray de shape (n_amostras - n_steps, n_steps)
        Matrizes de entrada para o LSTM, onde cada linha corresponde a uma janela de tamanho n_steps.
    y : np.ndarray de shape (n_amostras - n_steps,)
        Vetor de saída contendo o valor imediatamente seguinte a cada janela de entrada.
    """
    X, y = [], []
    for i in range(n_steps, len(serie)):
        X.append(serie[i - n_steps:i])
        y.append(serie[i])
    return np.array(X), np.array(y)


In [ ]:
def prever_demanda_meses(df, n_meses=12):
    """
    Treina um modelo de rede neural LSTM sobre o histórico geral de consumo mensal
    e gera previsões para os próximos meses.

    Descrição:
    - O DataFrame é agrupado por mês de lançamento, somando as quantidades movimentadas.
    - A série temporal é normalizada entre 0 e 1 usando MinMaxScaler.
    - Utiliza-se uma janela de 192 meses como entrada para o modelo (n_steps=192).
    - Um modelo LSTM é treinado para aprender o padrão do consumo mensal.
    - Após o treinamento, são geradas previsões para os próximos 'n_meses' meses.
    - As previsões são revertidas à escala original e plotadas junto ao histórico recente.

    Parâmetros:
    - df (DataFrame): Dados de entrada contendo pelo menos as colunas:
        - 'Data de lançamento' (datas dos registros)
        - 'Qtd.  UM registro' (quantidade movimentada)
    - n_meses (int, opcional): Número de meses futuros que deseja prever. Padrão é 12.

    Retorno:
    - DataFrame com as colunas:
        - 'Mês Previsto': Período previsto no formato 'YYYY-MM'
        - 'Qtd Prevista': Quantidade estimada para o respectivo mês

    Observações:
    - A função exibe um gráfico com o histórico recente (a partir de 2023) e as previsões para facilitar a análise visual.
    - É recomendado que o DataFrame de entrada contenha pelo menos 200 meses de histórico para garantir treinamento adequado.
    """


    df = df.rename(columns=lambda x: x.strip())
    df = df.rename(columns={'Qtd.  UM registro': 'Qtd'})

    serie_mensal = df.groupby('Data de lançamento')['Qtd'].sum().sort_index()

    scaler = MinMaxScaler()
    serie_normalizada = scaler.fit_transform(serie_mensal.values.reshape(-1, 1))

    n_steps = 192
    X, y = preparar_dados_lstm(serie_normalizada, n_steps)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=400, verbose=0)

    previsoes = []
    ultimos_passos = serie_normalizada[-n_steps:]

    for _ in range(n_meses):
        entrada = ultimos_passos.reshape((1, n_steps, 1))
        pred = model.predict(entrada, verbose=0)
        previsoes.append(pred[0][0])
        ultimos_passos = np.append(ultimos_passos[1:], pred, axis=0)

    previsoes_reais = scaler.inverse_transform(np.array(previsoes).reshape(-1, 1)).flatten()

    ultimo_mes = pd.to_datetime(serie_mensal.index[-1] + "-01")
    proximos_meses = pd.date_range(start=ultimo_mes + pd.offsets.MonthBegin(), periods=n_meses, freq='MS').strftime('%Y-%m')

    tabela_previsoes = pd.DataFrame({
        'Mês Previsto': proximos_meses,
        'Qtd Prevista': previsoes_reais
    })

    serie_plot = pd.concat([serie_mensal[serie_mensal.index >= '2023-01'], 
                            pd.Series(previsoes_reais, index=proximos_meses)])
    serie_plot = serie_plot[serie_plot.index >= '2024-01']

    plt.figure(figsize=(10, 5))
    plt.plot(serie_plot.index, serie_plot.values, marker='o', linestyle='-')
    plt.title(f'Previsão de Demanda para 2024')
    plt.xlabel('Mês')
    plt.ylabel('Quantidade')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return tabela_previsoes


In [ ]:


def prever_demanda_lstm_por_material_az(df, materiais_az, n_steps=12, n_meses_prever=6, n_epochs=300):
    """
Treina e gera previsões de demanda usando um modelo LSTM separado para cada material listado.

Descrição:
- Para cada material informado, a função constrói uma série temporal mensal a partir do histórico de vendas/consumo.
- Em seguida, prepara os dados em formato supervisionado (usando janelas de n_steps meses anteriores para prever o mês seguinte).
- Treina um modelo de rede neural LSTM individualmente para cada material, utilizando apenas seus próprios dados históricos.
- Após o treinamento, gera previsões para os próximos meses e plota um gráfico com:
  - Linha histórica (consumo real)
  - Linha prevista (projeção da IA)

Parâmetros:
- df (DataFrame): DataFrame contendo os dados históricos, obrigatoriamente com as colunas:
    - 'Material': código ou nome do material.
    - 'Data de lançamento': data do registro de movimentação (estoque/consumo).
    - 'Qtd. UM registro': quantidade movimentada nesse lançamento.
- materiais_az (list ou array): Lista de materiais que serão previstos individualmente (ex: ['A33K133', 'PAPEL COUCHE']).
- n_steps (int, opcional): Número de meses consecutivos usados como janela de entrada para a rede LSTM (padrão = 12).
- n_meses_prever (int, opcional): Quantidade de meses futuros a serem previstos após o fim da série histórica (padrão = 6).
- n_epochs (int, opcional): Número de épocas de treinamento do modelo LSTM para cada material (padrão = 300).

Notas:
- Materiais com histórico insuficiente (menos de `n_steps + 1` meses) serão automaticamente ignorados.
- A normalização dos dados é feita internamente usando MinMaxScaler para otimizar o desempenho do LSTM.
- Cada material é tratado de forma independente, com um modelo próprio e previsões específicas.
    """

    # Garantir formato de data e criar coluna AnoMes
    df = df.copy()
    df['Data de lançamento'] = pd.to_datetime(df['Data de lançamento'])
    df['AnoMes'] = df['Data de lançamento'].dt.to_period('M')

    def preparar_dados_lstm(serie, n_steps):
        X, y = [], []
        for i in range(n_steps, len(serie)):
            X.append(serie[i - n_steps:i])
            y.append(serie[i])
        return np.array(X), np.array(y)

    for mat in materiais_az:
        dados_mat = df[df['Material'] == mat]
        serie_mensal = dados_mat.groupby('AnoMes')['Qtd.  UM registro'].sum().sort_index()

        if len(serie_mensal) < n_steps + 1:
            print(f"Pulando {mat} - poucos dados ({len(serie_mensal)} meses)")
            continue

        scaler = MinMaxScaler()
        serie_normalizada = scaler.fit_transform(serie_mensal.values.reshape(-1, 1))

        X, y = preparar_dados_lstm(serie_normalizada, n_steps)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        model.fit(X, y, epochs=n_epochs, verbose=0)

        previsoes = []
        entrada = serie_normalizada[-n_steps:]

        for _ in range(n_meses_prever):
            entrada_reshaped = entrada.reshape((1, n_steps, 1))
            pred = model.predict(entrada_reshaped, verbose=0)[0][0]
            previsoes.append(pred)
            entrada = np.append(entrada[1:], [[pred]], axis=0)

        previsoes_reais = scaler.inverse_transform(np.array(previsoes).reshape(-1, 1)).flatten()

        plt.figure(figsize=(10, 4))
        plt.plot(serie_mensal.index.astype(str), serie_mensal.values, label='Histórico')
        proximos_meses = pd.date_range(start=serie_mensal.index[-1].to_timestamp() + pd.offsets.MonthBegin(),
                                       periods=n_meses_prever, freq='MS').strftime('%Y-%m')
        plt.plot(proximos_meses, previsoes_reais, label='Previsão', marker='o')
        plt.title(f'Previsão para {mat} (Classe AZ)')
        plt.xlabel('Mês')
        plt.ylabel('Quantidade')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
def tratar_dados(df: pd.DataFrame) -> pd.DataFrame:
    df['Data de lançamento'] = pd.to_datetime(df['Data de lançamento'], errors='coerce')
    
    df['Data de lançamento'] = df['Data de lançamento'].dt.to_period('M').astype(str)

    df['Qtd.  UM registro'] = df['Qtd.  UM registro'] * -1

    return df


### Limpando

In [ ]:
pccc_20_25 = carregar_arquivo_excel(caminhos['caminho_pccc_20_25'])
pccc_15_19 = carregar_arquivo_excel(caminhos['caminho_pccc_15_19'])
pccc_09_14 = carregar_arquivo_excel(caminhos['caminho_pccc_09_14'])
pccc_06_08 = carregar_arquivo_excel(caminhos['caminho_pccc_06_08'])

In [ ]:
pccc_20_25.columns

In [ ]:
colunas_remover = ['Centro', 'Depósito', 'Tipo de movimento',
       'Estoque especial', 'Doc.material', 'Item doc.material','UM registro', 'Cliente',
       'Fornecedor', 'Nome 1', 'Canal', 'Classificação']



pccc_20_25 = limpar_colunas(pccc_20_25, colunas_remover, nova_index='Material')
pccc_15_19 = limpar_colunas(pccc_15_19, colunas_remover, nova_index='Material')
pccc_09_14 = limpar_colunas(pccc_09_14, colunas_remover, nova_index='Material')
pccc_06_08 = limpar_colunas(pccc_06_08, colunas_remover, nova_index='Material')




In [ ]:
pccc_total = pd.concat([pccc_20_25, pccc_15_19, pccc_09_14, pccc_06_08], ignore_index=True)

In [ ]:
pccc_total = tratar_dados(pccc_total)

In [ ]:
pccc_total.columns

In [ ]:
pccc_ate_2023 = pccc_total[pccc_total['Data de lançamento'] < '2024-01']


In [ ]:
pccc_total

In [ ]:
pccc_ate_2023

# consolidado

In [ ]:
tabela_2024 = prever_demanda_meses(pccc_ate_2023, n_meses=12)
print(tabela_2024.to_string(index=False))

In [ ]:
pccc_total.columns = pccc_total.columns.str.strip()
pccc_total = pccc_total.rename(columns={'Qtd.  UM registro': 'Qtd'})

pccc_2024 = pccc_total[pccc_total['Data de lançamento'].str.startswith('2024')]

tabela_2024 = pccc_2024.groupby('Data de lançamento')['Qtd'].sum().reset_index()
tabela_2024.columns = ['Mês', 'Quantidade Total']

print("\n Valores já registrados em 2024 no pccc_total:\n")
print(tabela_2024.to_string(index=False))


### Comparação de tabelas

In [ ]:
previsao_2024 = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Prevista': [
        18919.91, 19154.69, 19202.46, 19250.48, 19288.11, 19320.03,
        19348.03, 19373.22, 19396.05, 19417.14, 19436.68, 19454.69
    ]
})

real_2024 = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Real': [
        16951, 15910, 20799, 24695, 21079, 18761,
        23163, 18830, 21348, 25105, 21031, 37398
    ]
})

comparacao = pd.merge(previsao_2024, real_2024, on='Mês')
comparacao['Erro Absoluto'] = abs(comparacao['Qtd Prevista'] - comparacao['Qtd Real'])
comparacao['Erro (%)'] = 100 * comparacao['Erro Absoluto'] / comparacao['Qtd Real']

mape = comparacao['Erro (%)'].mean()

print("\n Comparação entre Previsão e Real (2024)(n=12, epo =400):\n")
print(comparacao.to_string(index=False))
print(f"\n MAPE (Erro Percentual Médio): {mape:.2f}%")


In [ ]:

# Previsões geradas
previsao_2024 = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Prevista': [
        18919.91, 19154.69, 19202.46, 19250.48, 19288.11, 19320.03,
        19348.03, 19373.22, 19396.05, 19417.14, 19436.68, 19454.69
    ]
})

# Valores reais extraídos do pccc_total
real_2024 = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Real': [
        16951, 15910, 20799, 24695, 21079, 18761,
        23163, 18830, 21348, 25105, 21031, 37398
    ]
})

comparacao = pd.merge(previsao_2024, real_2024, on='Mês')
comparacao['Erro Absoluto'] = abs(comparacao['Qtd Prevista'] - comparacao['Qtd Real'])
comparacao['Erro (%)'] = 100 * comparacao['Erro Absoluto'] / comparacao['Qtd Real']

mape = comparacao['Erro (%)'].mean()

# Mostrar
print("\n Comparação entre Previsão e Real (2024)  (n_steps = 72,epochs=400):\n")
print(comparacao.to_string(index=False))
print(f"\n MAPE (Erro Percentual Médio): {mape:.2f}%")

In [ ]:
previsao_2024 = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Prevista': [
        19973.26, 20298.67, 20643.86, 21008.06, 21390.83, 21792.33,
        22212.87, 22653.00, 23113.56, 23595.57, 24100.38, 24629.71
    ]
})

# Valores reais extraídos do pccc_total
real_2024 = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Real': [
        16951, 15910, 20799, 24695, 21079, 18761,
        23163, 18830, 21348, 25105, 21031, 37398
    ]
})

# Unir as tabelas
comparacao = pd.merge(previsao_2024, real_2024, on='Mês')
comparacao['Erro Absoluto'] = abs(comparacao['Qtd Prevista'] - comparacao['Qtd Real'])
comparacao['Erro (%)'] = 100 * comparacao['Erro Absoluto'] / comparacao['Qtd Real']

# Calcular MAPE
mape = comparacao['Erro (%)'].mean()

# Mostrar
print("\n Comparação entre Previsão e Real (2024)  (n_steps = 194,epochs=400):\n")
print(comparacao.to_string(index=False))
print(f"\n MAPE (Erro Percentual Médio): {mape:.2f}%")

# canal

In [ ]:
pccc_20_25 = carregar_arquivo_excel(caminhos['caminho_pccc_20_25'])
pccc_15_19 = carregar_arquivo_excel(caminhos['caminho_pccc_15_19'])
pccc_09_14 = carregar_arquivo_excel(caminhos['caminho_pccc_09_14'])
pccc_06_08 = carregar_arquivo_excel(caminhos['caminho_pccc_06_08'])

In [ ]:
pccc_total = pd.concat([pccc_20_25, pccc_15_19,pccc_09_14,pccc_06_08])

In [ ]:
pccc_total.columns

In [ ]:
pccc_total['Canal'].unique()

In [ ]:
pccc_venda_indireta = pccc_total[pccc_total['Canal'].str.upper().str.strip() == 'VENDA INDIRETA']
pccc_venda_direta = pccc_total[pccc_total['Canal'].str.upper().str.strip() == 'VENDA DIRETA']


In [ ]:
pccc_venda_indireta.columns

In [ ]:
colunas_remover = ['Centro', 'Depósito', 'Tipo de movimento',
       'Estoque especial', 'Doc.material', 'Item doc.material','UM registro', 'Cliente',
       'Fornecedor', 'Nome 1', 'Classificação']

pccc_venda_indireta = limpar_colunas(pccc_venda_indireta, colunas_remover, nova_index='Material')
pccc_venda_direta = limpar_colunas(pccc_venda_direta, colunas_remover, nova_index='Material')

In [ ]:
pccc_venda_indireta

In [ ]:
pccc_venda_indireta = tratar_dados(pccc_venda_indireta)
pccc_venda_direta = tratar_dados(pccc_venda_direta)

## direta

In [ ]:
df_lstm_direta = pccc_venda_direta.groupby(['Canal', 'Data de lançamento'])['Qtd.  UM registro'].sum().reset_index()


In [ ]:
df_lstm_direta

In [ ]:
df_lstm_direta = df_lstm_direta[df_lstm_direta['Data de lançamento'] <= '2023-12']
df_lstm_direta

In [ ]:
prever_demanda_meses(df_lstm_direta, n_meses=12)

In [ ]:
pccc_venda_direta_2024 = pccc_venda_direta[pccc_venda_direta['Data de lançamento'].str.startswith('2024')]

In [ ]:
tabela_venda_direta_2024 = (
    pccc_venda_direta_2024
    .groupby('Data de lançamento')['Qtd.  UM registro']
    .sum()
    .reset_index()
)

# Renomear colunas pra ficar bonitinho
tabela_venda_direta_2024.columns = ['Mês', 'Quantidade Total']


In [ ]:
tabela_venda_direta_2024

### Analisando resultados

In [ ]:

real_venda_direta_2024 = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Real': [
        5365, 5276, 7077, 7062, 5387, 5780,
        6705, 7058, 6282, 6961, 6644, 6875
    ]
})

previsao_venda_direta_2024 = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Prevista': [
        6552.55, 6542.64, 6534.02, 6526.04, 6518.63, 6511.72,
        6505.28, 6499.25, 6493.62, 6488.34, 6483.39, 6478.75
    ]
})

comparacao = pd.merge(real_venda_direta_2024, previsao_venda_direta_2024, on='Mês')
comparacao['Erro Absoluto'] = abs(comparacao['Qtd Prevista'] - comparacao['Qtd Real'])
comparacao['Erro (%)'] = 100 * comparacao['Erro Absoluto'] / comparacao['Qtd Real']

mape = comparacao['Erro (%)'].mean()

print("\n Comparação VENDA DIRETA 2024:\n")
print(comparacao.to_string(index=False))
print(f"\n MAPE (Erro Percentual Médio): {mape:.2f}%")

## indireta

In [ ]:
df_lstm_indireta = pccc_venda_indireta.groupby(['Canal', 'Data de lançamento'])['Qtd.  UM registro'].sum().reset_index()


In [ ]:
df_lstm_indireta = df_lstm_indireta[df_lstm_indireta['Data de lançamento'] <= '2023-12']


In [ ]:
df_lstm_indireta

In [ ]:
prever_demanda_meses(df_lstm_indireta, n_meses=12)

In [ ]:
pccc_venda_indireta_2024 = pccc_venda_indireta[pccc_venda_indireta['Data de lançamento'].str.startswith('2024')]

tabela_venda_indireta_2024 = (
    pccc_venda_indireta_2024
    .groupby('Data de lançamento')['Qtd.  UM registro']
    .sum()
    .reset_index()
)

# Renomear colunas pra ficar bonitinho
tabela_venda_indireta_2024.columns = ['Mês', 'Quantidade Total']

In [ ]:
tabela_venda_indireta_2024

### Analisando resultados

In [ ]:

real_2024_indireta = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Real': [
        11538, 10612, 13654, 15540, 15659, 12850,
        16388, 11727, 15035, 18086, 11602, 30453
    ]
})

previsto_2024_indireta = pd.DataFrame({
    'Mês': [
        '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
        '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
    ],
    'Qtd Prevista': [
        13020.82, 13276.03, 13542.58, 13823.19, 14119.71, 14434.00,
        14768.21, 15124.79, 15506.56, 15916.78, 16359.24, 16838.38
    ]
})

comparacao = pd.merge(real_2024_indireta, previsto_2024_indireta, on='Mês')
comparacao['Erro Absoluto'] = abs(comparacao['Qtd Real'] - comparacao['Qtd Prevista'])
comparacao['Erro (%)'] = 100 * comparacao['Erro Absoluto'] / comparacao['Qtd Real']

mape = comparacao['Erro (%)'].mean()

print("\n Comparação VENDA INDIRETA 2024:\n")
print(comparacao.to_string(index=False))
print(f"\n MAPE (Erro Percentual Médio): {mape:.2f}%")


# material

## abc/xyz 19 anos

In [ ]:
pccc_20_25 = carregar_arquivo_excel(caminhos['caminho_pccc_20_25'])
pccc_15_19 = carregar_arquivo_excel(caminhos['caminho_pccc_15_19'])
pccc_09_14 = carregar_arquivo_excel(caminhos['caminho_pccc_09_14'])
pccc_06_08 = carregar_arquivo_excel(caminhos['caminho_pccc_06_08'])

In [ ]:
pccc_total = pd.concat([pccc_20_25, pccc_15_19,pccc_09_14,pccc_06_08])

In [ ]:
pccc_total

In [ ]:
pccc_total = tratar_dados(pccc_total)

In [ ]:
pccc_total

In [ ]:
pccc_total.columns

In [ ]:
pccc_total['Data de lançamento'] = pd.to_datetime(pccc_total['Data de lançamento'])
pccc_total['AnoMes'] = pccc_total['Data de lançamento'].dt.to_period('M')

demanda_mensal = pccc_total.groupby(['Material', 'AnoMes'])['Qtd.  UM registro'].sum().unstack(fill_value=0)


In [ ]:
total_por_material = demanda_mensal.sum(axis=1)
percentual = total_por_material / total_por_material.sum()

# Ordenar do maior pro menor
df_abc = percentual.sort_values(ascending=False).cumsum()

# Criar categoria ABC
def classificar_abc(x):
    if x <= 0.8:
        return 'A'
    elif x <= 0.95:
        return 'B'
    else:
        return 'C'

classe_abc = df_abc.apply(classificar_abc)


In [ ]:
media = demanda_mensal.mean(axis=1)
desvio = demanda_mensal.std(axis=1)
cv = (desvio / media).fillna(0)

def classificar_xyz(x):
    if x <= 0.5:
        return 'X'
    elif x <= 1:
        return 'Y'
    else:
        return 'Z'

classe_xyz = cv.apply(classificar_xyz)


In [ ]:
df_classificacao = pd.DataFrame({
    'Total Movimentado': total_por_material,
    'Classe ABC': classe_abc,
    'Coef. Variação': cv,
    'Classe XYZ': classe_xyz
})

df_classificacao['Classe ABC/XYZ'] = df_classificacao['Classe ABC'] + df_classificacao['Classe XYZ']


In [ ]:
df_classificacao

In [ ]:
pccc_total

In [ ]:
abc_labels = ['A', 'B', 'C']
xyz_labels = ['X', 'Y', 'Z']

tabela = pd.crosstab(df_classificacao['Classe ABC'], df_classificacao['Classe XYZ'])
tabela = tabela.reindex(index=abc_labels, columns=xyz_labels, fill_value=0)
data = tabela.values

fig, ax = plt.subplots(figsize=(8, 6))

for i in range(len(abc_labels)):
    for j in range(len(xyz_labels)):
        valor = data[i, j]
        ax.text(j, i, str(valor), va='center', ha='center', fontsize=14, fontweight='bold', color='black')
        ax.add_patch(plt.Rectangle((j - 0.5, i - 0.5), 1, 1, fill=False, edgecolor='black', lw=1))

ax.set_xticks(np.arange(len(xyz_labels)))
ax.set_xticklabels(xyz_labels)
ax.set_yticks(np.arange(len(abc_labels)))
ax.set_yticklabels(abc_labels)
ax.set_xlabel('Classe XYZ')
ax.set_ylabel('Classe ABC')
ax.set_title('Quadrante ABC/XYZ - Distribuição de Materiais')

ax.set_xlim(-0.5, len(xyz_labels) - 0.5)
ax.set_ylim(len(abc_labels) - 0.5, -0.5)
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

In [ ]:
pccc_az = df_classificacao[df_classificacao['Classe ABC/XYZ'] == 'AZ'].reset_index()
lista_materiais_az = pccc_az['Material'].unique().tolist()


In [ ]:
lista_materiais_az

In [ ]:
pccc_total_az = pccc_total[pccc_total['Material'].isin(lista_materiais_az)]

In [ ]:
pccc_total_az

In [ ]:
pccc_total_az_filtrado = pccc_total_az[pccc_total_az['AnoMes'] < '2024-01']

In [ ]:
pccc_total_az_filtrado

In [ ]:
#prever_demanda_lstm_por_material_az(df=pccc_total_az, materiais_az=lista_materiais_az)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

def prever_demanda_lstm_por_material_az_sem_plot(df, materiais_az, n_steps=12, n_meses_prever=6, n_epochs=300):
    """
    Treina um modelo LSTM individual por material da classe AZ e retorna as previsões em um DataFrame.

    Parâmetros:
    - df: DataFrame com ['Material', 'Data de lançamento', 'Qtd. UM registro']
    - materiais_az: lista de códigos de material
    - n_steps: meses usados como janela para treino
    - n_meses_prever: meses a prever
    - n_epochs: número de épocas do modelo

    Retorna:
    - DataFrame com colunas: ['Material', 'Mês Previsto', 'Qtd Prevista']
    """
    resultados = []

    df = df.copy()
    df['Data de lançamento'] = pd.to_datetime(df['Data de lançamento'])
    df['AnoMes'] = df['Data de lançamento'].dt.to_period('M')

    def preparar_dados_lstm(serie, n_steps):
        X, y = [], []
        for i in range(n_steps, len(serie)):
            X.append(serie[i - n_steps:i])
            y.append(serie[i])
        return np.array(X), np.array(y)

    for mat in materiais_az:
        dados_mat = df[df['Material'] == mat]
        serie_mensal = dados_mat.groupby('AnoMes')['Qtd.  UM registro'].sum().sort_index()

        if len(serie_mensal) < n_steps + 1:
            print(f"Pulando {mat} - poucos dados ({len(serie_mensal)} meses)")
            continue

        scaler = MinMaxScaler()
        serie_normalizada = scaler.fit_transform(serie_mensal.values.reshape(-1, 1))
        X, y = preparar_dados_lstm(serie_normalizada, n_steps)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        model = Sequential()
        model.add(LSTM(50, activation='tanh', return_sequences=True, input_shape=(n_steps, 1)))
        model.add(LSTM(50, activation='tanh'))  # segunda camada
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        model.fit(X, y, epochs=n_epochs, verbose=0)

        previsoes = []
        entrada = serie_normalizada[-n_steps:]

        for _ in range(n_meses_prever):
            entrada_reshaped = entrada.reshape((1, n_steps, 1))
            pred = model.predict(entrada_reshaped, verbose=0)[0][0]
            previsoes.append(pred)
            entrada = np.append(entrada[1:], [[pred]], axis=0)

        previsoes_reais = scaler.inverse_transform(np.array(previsoes).reshape(-1, 1)).flatten()
        meses_futuros = pd.date_range(
            start=serie_mensal.index[-1].to_timestamp() + pd.offsets.MonthBegin(),
            periods=n_meses_prever, freq='MS'
        ).strftime('%Y-%m')

        for mes, qtd in zip(meses_futuros, previsoes_reais):
            resultados.append({
                'Material': mat,
                'Mês Previsto': mes,
                'Qtd Prevista': qtd
            })

    return pd.DataFrame(resultados)


In [ ]:
previsoes_df = prever_demanda_lstm_por_material_az_sem_plot(
    df=pccc_total_az_filtrado,
    materiais_az=lista_materiais_az,
    n_steps=48,
    n_meses_prever=6,
    n_epochs=400
)

# Ver resultado
print(previsoes_df.head())

# (Opcional) salvar em Excel
# previsoes_df.to_excel('previsoes_materiais_az.xlsx', index=False)
#         model = Sequential()
#       model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
#        model.add(Dense(1))
#        model.compile(optimizer='adam', loss='mse')
#        model.fit(X, y, epochs=n_epochs, verbose=0)

In [ ]:
previsoes_df 

In [ ]:
# previsoes_df.to_excel('previsoes_materiais_az.xlsx', index=False)


In [ ]:
pccc_total_az

In [ ]:
# 1. Garantir que as datas estão no mesmo formato
pccc_total['Data de lançamento'] = pd.to_datetime(pccc_total['Data de lançamento'])
pccc_total['AnoMes'] = pccc_total['Data de lançamento'].dt.to_period('M').astype(str)

# 2. Se ainda não estiver, garantir que Mês Previsto também está como string YYYY-MM
previsoes_df['Mês Previsto'] = previsoes_df['Mês Previsto'].astype(str)

# 3. Agrupar o total real por Material e AnoMes
saidas_reais = (
    pccc_total.groupby(['Material', 'AnoMes'])['Qtd.  UM registro']
    .sum()
    .reset_index()
    .rename(columns={'Qtd. UM registro': 'Qtd Real'})
)

# 4. Juntar a previsão com o real
previsoes_com_reais = previsoes_df.merge(
    saidas_reais,
    left_on=['Material', 'Mês Previsto'],
    right_on=['Material', 'AnoMes'],
    how='left'
).drop(columns=['AnoMes'])


In [ ]:
previsoes_com_reais

In [ ]:
previsoes_com_reais.to_excel('previsoes_com_reais.xlsx', index=False)


In [ ]:
print(previsoes_com_reais.columns)


In [ ]:
# Renomear a coluna para facilitar o uso
previsoes_com_reais = previsoes_com_reais.rename(columns={'Qtd.  UM registro': 'Qtd_Real'})

# Substituir zeros por NaN para evitar divisão por zero
previsoes_com_reais['Qtd_Real'] = previsoes_com_reais['Qtd_Real'].replace(0, np.nan)

# Calcular erro absoluto
previsoes_com_reais['Erro_Absoluto'] = (previsoes_com_reais['Qtd_Real'] - previsoes_com_reais['Qtd_Prevista']).abs()

# Calcular erro percentual
previsoes_com_reais['Erro_%'] = (previsoes_com_reais['Erro_Absoluto'] / previsoes_com_reais['Qtd_Real']) * 100


In [ ]:
previsoes_com_reais

In [ ]:
# Agrupar por Material e calcular a média do erro percentual
media_erro_por_material = (
    previsoes_com_reais
    .groupby('Material')['Erro_%']
    .mean()
    .reset_index()
    .rename(columns={'Erro_%': 'Erro Médio (%)'})
    .sort_values(by='Erro Médio (%)', ascending=False)
)


In [ ]:
media_erro_por_material

In [ ]:
#previsoes_com_reais.to_excel('previsoes_com_reais.xlsx', index=False)
#media_erro_por_material.to_excel('media_erro_por_material.xlsx', index=False)

In [ ]:
materiais_confiaveis = media_erro_por_material[media_erro_por_material['Erro Médio (%)'] <= 30]

In [ ]:
materiais_confiaveis

In [ ]:
previsoes_df = prever_demanda_lstm_por_material_az_sem_plot(
    df=pccc_total_az_filtrado,
    materiais_az=lista_materiais_az,
    n_steps=72,
    n_meses_prever=6,
    n_epochs=300
)

# Ver resultado
previsoes_df

In [ ]:
# 1. Garantir que as datas estão no mesmo formato
pccc_total['Data de lançamento'] = pd.to_datetime(pccc_total['Data de lançamento'])
pccc_total['AnoMes'] = pccc_total['Data de lançamento'].dt.to_period('M').astype(str)

# 2. Se ainda não estiver, garantir que Mês Previsto também está como string YYYY-MM
previsoes_df['Mês Previsto'] = previsoes_df['Mês Previsto'].astype(str)

# 3. Agrupar o total real por Material e AnoMes
saidas_reais = (
    pccc_total.groupby(['Material', 'AnoMes'])['Qtd.  UM registro']
    .sum()
    .reset_index()
    .rename(columns={'Qtd. UM registro': 'Qtd Real'})
)

# 4. Juntar a previsão com o real
previsoes_com_reais = previsoes_df.merge(
    saidas_reais,
    left_on=['Material', 'Mês Previsto'],
    right_on=['Material', 'AnoMes'],
    how='left'
).drop(columns=['AnoMes'])

In [ ]:
previsoes_com_reais

In [ ]:
# Renomear a coluna para facilitar o uso

# Substituir zeros por NaN para evitar divisão por zero
previsoes_com_reais['Qtd_Real'] = previsoes_com_reais['Qtd_Real'].replace(0, np.nan)

# Calcular erro absoluto
previsoes_com_reais['Erro_Absoluto'] = (previsoes_com_reais['Qtd_Real'] - previsoes_com_reais['Qtd Prevista']).abs()

# Calcular erro percentual
previsoes_com_reais['Erro_%'] = (previsoes_com_reais['Erro_Absoluto'] / previsoes_com_reais['Qtd_Real']) * 100

In [ ]:
previsoes_com_reais

In [ ]:
# Agrupar por Material e calcular a média do erro percentual
media_erro_por_material = (
    previsoes_com_reais
    .groupby('Material')['Erro_%']
    .mean()
    .reset_index()
    .rename(columns={'Erro_%': 'Erro Médio (%)'})
    .sort_values(by='Erro Médio (%)', ascending=False)
)


In [ ]:
media_erro_por_material

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(media_erro_por_material
)


## abc/xyz usando 3 anos

In [ ]:
pccc_total

In [ ]:
pccc_total_filtrado = pccc_total[pccc_total['AnoMes'] > '2019-12']

In [ ]:
pccc_total_filtrado

In [ ]:
pccc_total_filtrado['Data de lançamento'] = pd.to_datetime(pccc_total_filtrado['Data de lançamento'])
pccc_total_filtrado['AnoMes'] = pccc_total_filtrado['Data de lançamento'].dt.to_period('M')

demanda_mensal = pccc_total_filtrado.groupby(['Material', 'AnoMes'])['Qtd.  UM registro'].sum().unstack(fill_value=0)

In [ ]:
total_por_material = demanda_mensal.sum(axis=1)
percentual = total_por_material / total_por_material.sum()

df_abc = percentual.sort_values(ascending=False).cumsum()

def classificar_abc(x):
    if x <= 0.8:
        return 'A'
    elif x <= 0.95:
        return 'B'
    else:
        return 'C'

classe_abc = df_abc.apply(classificar_abc)


In [ ]:
media = demanda_mensal.mean(axis=1)
desvio = demanda_mensal.std(axis=1)
cv = (desvio / media).fillna(0)

def classificar_xyz(x):
    if x <= 0.5:
        return 'X'
    elif x <= 1:
        return 'Y'
    else:
        return 'Z'

classe_xyz = cv.apply(classificar_xyz)

In [ ]:
df_classificacao = pd.DataFrame({
    'Total Movimentado': total_por_material,
    'Classe ABC': classe_abc,
    'Coef. Variação': cv,
    'Classe XYZ': classe_xyz
})

df_classificacao['Classe ABC/XYZ'] = df_classificacao['Classe ABC'] + df_classificacao['Classe XYZ']


In [ ]:
abc_labels = ['A', 'B', 'C']
xyz_labels = ['X', 'Y', 'Z']

tabela = pd.crosstab(df_classificacao['Classe ABC'], df_classificacao['Classe XYZ'])
tabela = tabela.reindex(index=abc_labels, columns=xyz_labels, fill_value=0)
data = tabela.values

fig, ax = plt.subplots(figsize=(8, 6))

for i in range(len(abc_labels)):
    for j in range(len(xyz_labels)):
        valor = data[i, j]
        ax.text(j, i, str(valor), va='center', ha='center', fontsize=14, fontweight='bold', color='black')
        ax.add_patch(plt.Rectangle((j - 0.5, i - 0.5), 1, 1, fill=False, edgecolor='black', lw=1))

ax.set_xticks(np.arange(len(xyz_labels)))
ax.set_xticklabels(xyz_labels)
ax.set_yticks(np.arange(len(abc_labels)))
ax.set_yticklabels(abc_labels)
ax.set_xlabel('Classe XYZ')
ax.set_ylabel('Classe ABC')
ax.set_title('Quadrante ABC/XYZ - Distribuição de Materiais')

ax.set_xlim(-0.5, len(xyz_labels) - 0.5)
ax.set_ylim(len(abc_labels) - 0.5, -0.5)
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

In [ ]:
pccc_az = df_classificacao[df_classificacao['Classe ABC/XYZ'] == 'AZ'].reset_index()
lista_materiais = pccc_az['Material'].unique().tolist()

In [ ]:
lista_materiais

In [ ]:
pccc_total_filtrado_materiais = pccc_total[pccc_total['Material'].isin(lista_materiais)]

In [ ]:
pccc_total_filtrado_materiais

In [ ]:
pccc_total_filtrado_materiais_1 = pccc_total_filtrado_materiais[pccc_total_filtrado_materiais['AnoMes'] < '2023-12']


In [ ]:
previsoes_df = prever_demanda_lstm_por_material_az_sem_plot(
    df=pccc_total_filtrado_materiais_1,
    materiais_az=lista_materiais,
    n_steps=33,
    n_meses_prever=12,
    n_epochs=300
)

previsoes_df

In [ ]:
previsoes_df.columns

In [ ]:
pccc_total_filtrado_materiais.columns

In [ ]:
previsoes_df
valores_unicos = previsoes_df['Material'].unique()


In [ ]:
valores_unicos

In [ ]:

previsoes_2024 = previsoes_df[previsoes_df['Mês Previsto'] >= '2024-01']

dados_reais_2024 = pccc_total_filtrado_materiais[pccc_total_filtrado_materiais['AnoMes'] >= '2024-01']

reais_por_mes = dados_reais_2024.groupby('AnoMes')['Qtd.  UM registro'].sum().reset_index()
reais_por_mes.rename(columns={'AnoMes': 'Mês', 'Qtd.  UM registro': 'Qtd Real'}, inplace=True)

previsoes_por_mes = previsoes_2024.groupby('Mês Previsto')['Qtd Prevista'].sum().reset_index()
previsoes_por_mes.rename(columns={'Mês Previsto': 'Mês'}, inplace=True)

comparacao = pd.merge(previsoes_por_mes, reais_por_mes, on='Mês', how='inner')

comparacao['Erro Absoluto'] = (comparacao['Qtd Prevista'] - comparacao['Qtd Real']).abs()
comparacao['Erro (%)'] = (comparacao['Erro Absoluto'] / comparacao['Qtd Real']) * 100



In [ ]:
comparacao

In [ ]:
previsoes_A33K133 = previsoes_df[(previsoes_df['Material'] == 'A33K133') & (previsoes_df['Mês Previsto'] >= '2024-01')]

dados_reais_A33K133 = pccc_total_filtrado_materiais[
    (pccc_total_filtrado_materiais['Material'] == 'A33K133') &
    (pccc_total_filtrado_materiais['AnoMes'] >= '2024-01')
]

reais_A33K133_por_mes = dados_reais_A33K133.groupby('AnoMes')['Qtd.  UM registro'].sum().reset_index()
reais_A33K133_por_mes.rename(columns={'AnoMes': 'Mês', 'Qtd.  UM registro': 'Qtd Real'}, inplace=True)

previsoes_A33K133_por_mes = previsoes_A33K133.groupby('Mês Previsto')['Qtd Prevista'].sum().reset_index()
previsoes_A33K133_por_mes.rename(columns={'Mês Previsto': 'Mês'}, inplace=True)

comparacao_A33K133 = pd.merge(previsoes_A33K133_por_mes, reais_A33K133_por_mes, on='Mês', how='inner')

comparacao_A33K133['Erro Absoluto'] = (comparacao_A33K133['Qtd Prevista'] - comparacao_A33K133['Qtd Real']).abs()
comparacao_A33K133['Erro (%)'] = (comparacao_A33K133['Erro Absoluto'] / comparacao_A33K133['Qtd Real']) * 100

media_erro_A33K133 = comparacao_A33K133['Erro (%)'].mean()




In [ ]:
comparacao_A33K133

In [ ]:
previsoes_AAJW131 = previsoes_df[(previsoes_df['Material'] == 'AAJW131') & (previsoes_df['Mês Previsto'] >= '2024-01')]

dados_reais_AAJW131 = pccc_total_filtrado_materiais[
    (pccc_total_filtrado_materiais['Material'] == 'AAJW131') &
    (pccc_total_filtrado_materiais['AnoMes'] >= '2024-01')
]

reais_AAJW131_por_mes = dados_reais_AAJW131.groupby('AnoMes')['Qtd.  UM registro'].sum().reset_index()
reais_AAJW131_por_mes.rename(columns={'AnoMes': 'Mês', 'Qtd.  UM registro': 'Qtd Real'}, inplace=True)

previsoes_AAJW131_por_mes = previsoes_AAJW131.groupby('Mês Previsto')['Qtd Prevista'].sum().reset_index()
previsoes_AAJW131_por_mes.rename(columns={'Mês Previsto': 'Mês'}, inplace=True)

comparacao_AAJW131 = pd.merge(previsoes_AAJW131_por_mes, reais_AAJW131_por_mes, on='Mês', how='inner')

comparacao_AAJW131['Erro Absoluto'] = (comparacao_AAJW131['Qtd Prevista'] - comparacao_AAJW131['Qtd Real']).abs()
comparacao_AAJW131['Erro (%)'] = (comparacao_AAJW131['Erro Absoluto'] / comparacao_AAJW131['Qtd Real']) * 100

media_erro_AAJW131 = comparacao_AAJW131['Erro (%)'].mean()


In [ ]:
comparacao_AAJW131


In [ ]:
media_erro_AAJW131

# explicaçao


##  **Modelo Utilizado: LSTM**

O modelo aplicado para a previsão de demanda foi a LSTM especializada em aprender padrões de séries temporais, como sazonalidade, tendência e flutuações históricas.

A LSTM é especialmente eficaz em séries temporais **com dependências de longo prazo**, pois mantém "memória" de padrões anteriores, ajustando sua previsão com base em janelas de tempo passadas (definidas por `n_steps`). Neste caso:

- Foram utilizados **`n_steps = 194`** , ou seja, o modelo aprendeu com **194 meses anteriores**
- Treinamento foi feito com **`epochs = 400`**, permitindo que o modelo ajustasse bem seus pesos

---

##  **Resultados e Análise das Previsões**

### 🔹 **Previsão Total (Todos os Canais)**
| MAPE: **13.85%** |
- Boa performance geral, com vários meses abaixo de 10% de erro
- Erros mais altos nos extremos (jan, fev, dez) indicam **sazonalidade forte não capturada**
- Dezembro tem erro alto (34%), comum em meses com picos de demanda não recorrentes

---

### 🔹 **VENDA DIRETA**
| MAPE: **10.36%** |
- Excelente desempenho: a maioria dos meses apresenta erros **abaixo de 10%**
- Apenas jan, fev e maio passaram de 20%, indicando variações atípicas ou rupturas no padrão
- Modelo claramente **aprendeu bem o comportamento do canal VENDA DIRETA**

---

### 🔹 **VENDA INDIRETA**
| MAPE: **17.64%** |
- Desempenho razoável, mas com **alguns meses críticos**:
  - Novembro: erro de 41%
  - Dezembro: erro de 44%
- O modelo teve dificuldade de capturar os picos (possivelmente ligados a promoções ou sazonalidades fortes)
- Apesar disso, vários meses (mar, jul, set) tiveram erro **abaixo de 10%**

---

##  Conclusão

- O uso de LSTM se mostrou **eficaz** na modelagem da demanda total e por canal.
- **VENDA DIRETA teve a menor margem de erro** e uma curva muito mais estável, indicando previsibilidade maior nesse segmento.
- **VENDA INDIRETA exige ajustes no modelo ou inclusão de variáveis externas**, como promoções ou sazonalidade explícita, para capturar os picos.
- A previsão agregada (todos os canais) apresentou **bom equilíbrio**, mesmo com outliers como dezembro.

